In [67]:
from sklearn.neighbors import KNeighborsRegressor
import pandas as pd
import math
import numpy as np

In [68]:
df = pd.read_csv("data/imports-85-clean.csv")
df = df.loc[df["price"] != '?']
df = df.astype({"price": float}, errors='raise')
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]

In [69]:
df.dtypes

symboling            float64
normalized-losses    float64
make                   int64
fuel-type              int64
aspiration             int64
num-of-doors           int64
body-style             int64
drive-wheels           int64
engine-location        int64
wheel-base           float64
length               float64
width                float64
height               float64
curb-weight          float64
engine-type            int64
num-of-cylinders       int64
engine-size          float64
fuel-system            int64
bore                 float64
stroke               float64
compression-ratio    float64
horsepower           float64
peak-rpm             float64
city-mpg             float64
highway-mpg          float64
price                float64
dtype: object

In [70]:
neigh = KNeighborsRegressor(n_neighbors=5)
x_train = train.drop(columns=["price"])
y_train = train["price"]
x_test = test.drop(columns=["price"])
y_test = test["price"]
neigh.fit(x_train, y_train)

KNeighborsRegressor()

In [77]:
y_pred = neigh.predict(x_train)